# Path of Exile game stats data eda

dates: 2022.05.17 ~ 

This file contains an analysis processing of the game 'Path of Exile' data.

###### Origins of datas from here: 
poe_stats : https://www.kaggle.com/datasets/gagazet/path-of-exile-league-statistic

Data contains stats of 59000 players, from 4th August of 2017 and before now.

---

특정 유저들을 골라내는 위주로 분석을 진행할 노트북입니다. 이름을 이용하거나, 트위치 아이디를 이용하는 방식입니다.



# Dataset and libraries 

## data and libraries import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
# import wquantiles

# from sklearn.utils import resample
# from scipy import stats
# from scipy.stats import trim_mean
# from statsmodels import robust
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
# from statsmodels.stats import power

df = pd.read_csv('poe_stats_2revisedrank.csv')

In [2]:
df.head(10)

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
5,2,False,False,SteelStreamerAdvantage,100,Deadeye,51acb8e6fac3cfaa7ff848151dad01b0a6f350e7525543...,4250334444,Steelmage,37,steelmage202,Hardcore Harbinger,1,0,0,1,1
6,2,False,False,Grom_WhyGodThreeCmWhy,100,Gladiator,5d8b80b2bdd30f3dba1036a6791187562758935e8d4723...,4250334444,rocklegend,22,gromick,SSF Harbinger HC,1,1,0,1,2
7,2,False,False,beobio_capitan,100,Inquisitor,65c58f9b186e1fbe9f927484f3f178cdb9ee0dc2427289...,4250334444,TishaVosemsot,40,beobio,SSF Harbinger,1,1,1,0,2
8,3,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
9,3,False,False,FutonBlewAway,100,Raider,46e56572ff6e403f45fac597240e13e9af225f06ca4e52...,4250334444,pleaGRC,40,NaN,SSF Harbinger,1,0,0,1,1


# About 'id'
---


In [3]:
# 고유값 확인

print(df.shape)
print(df['id'].nunique())

(59776, 17)
59776


# About 'name'
---

id는 고유하지만 name은 겹치는 것들이 분명 있었습니다. 캐릭터 name을 정하는데에 있어서 제한이 있는 것으로는 보이지 않습니다.




In [4]:
# 고유값 확인

df['name'].nunique()

58286

In [5]:
df['name'].sample(5)

10303          smurf_bf
39503        AnalHorror
1357          GutsMiner
53359    kaboutersylvia
29254       loverecraft
Name: name, dtype: object

이름에서 겹치는 것들은 무엇이 있지?

아이디를 지을 수 있는 규칙은?

겹치면서 트위치 아이디가 같은 사람들도 있나 ? 그러면 같은 사람으로 유추할 수 있지 않을까?

비슷한 아이디가 또 있지 않을까? 비슷한 아이디는 어떻게?


##### 가장 확실한 실마리는  'account'이다.

In [45]:
df['account'].value_counts().sort_values(ascending=False)

chuz101       30
HQuality      29
TundraDrum    25
Hyperized     25
Ardwage       23
              ..
Qoom           1
Migiri         1
Shakaja        1
Clocksimus     1
woworpoe       1
Name: account, Length: 37123, dtype: int64

## same name, different id


In [41]:
df[['account','name', 'id', 'twitch']].describe()

,account,name,id,twitch
count,59776,59776,59776,16300
unique,37123,58286,59776,9613
top,chuz101,_Flic_,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,thespunkylobster
freq,30,14,1,18


같은 유저가 다른 id를 만들어서 운영중이기도 할까?

In [40]:
df[['account','name', 'id', 'twitch']].groupby(by = 'name').count()

,account,id,twitch
name,,,
AAAAAAAAAAAAAMUTALA,1,1,0
AAAAAAAAAAAAyourself,1,1,1
AAAAAAAAAAHHHHH,1,1,0
AAAAAAAAASUNDERAAAAAAAA,1,1,1
AAAAAAAAAlIENNNNNNNNN,1,1,0
...,...,...,...
๒๏๏๒,1,1,1
๓๏๓,2,2,2
๖Neferpitou๙,1,1,0


In [17]:
df[['name', 'id']].groupby(['name'])['id'].count().sort_values(ascending=False).head(30)

name
_Flic_             14
TundraDrum         11
LordTemplar        10
kotlecio            9
SirLalli            9
dopenesh            8
Hellrazah           8
Znoz                8
Cya                 8
Ekainen             7
Flashow             7
HarbinGer_Meng      7
Lodystek            7
HarbinGer_Mango     7
xWolhaiksong        7
This                6
Ryacchi             6
nextinthelane       6
Xiberia             6
Kokkuwi             6
uranari             6
Khob                6
Vorkath             6
VACK_HHC            6
Pingeee             6
Elki                6
Cursecoin           6
gMax                6
OvarianPillager     6
Mimic_SSF           6
Name: id, dtype: int64

In [31]:
# name = '_Flic_' are 14.

df[df['name'] == '_Flic_']

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
27206,6815,True,False,_Flic_,72,Ascendant,67bcd417afe0b89cfc308b26bf590b511180130f680583...,455996734,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
28071,7032,True,False,_Flic_,72,Ascendant,7ec0dc750d81ebacf2ca2cce9879da1d801f0631444d25...,440263744,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
28913,7243,True,False,_Flic_,71,Ascendant,9bd5774ed0f816b13e97673498d13aa82cbda6ebcea9c2...,424709581,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
36978,9263,True,False,_Flic_,68,Ascendant,72e45259ccd8f39f0582fe90c4f518bcd612b38d591bd8...,314784438,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
40020,10024,True,False,_Flic_,66,Ascendant,4f1ccb10064cfb440ac865c3c2aaa658ecbcd581374b3e...,273456954,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
41317,10349,True,False,_Flic_,65,Ascendant,11b3c4652b8198d542f3da11b08d7775b3460937a188b6...,253743465,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
42076,10539,True,False,_Flic_,65,Ascendant,e946cd6ebc3e957f39860a7d7f39efc0b632512d5b4fe5...,240160428,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
42405,10622,True,False,_Flic_,65,Ascendant,0633f8211bb7fe74aa2fba4316ee68f76db8f013af796c...,233709956,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
44372,11116,True,False,_Flic_,63,Ascendant,0003c4ccac35b1f8c69a8b5b18701bf1a7310800ac1f56...,196299619,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
46567,11665,True,False,_Flic_,61,Ascendant,3eda4821da06924fe80ef166ff5f00fd639d4d0c611cf4...,166472598,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3


In [42]:
df[df['name'] == '_Flic_'].nunique()

rank             14
dead              1
online            1
name              1
level            10
class             1
id               14
experience       14
account           1
challenges        1
twitch            0
ladder            1
special_class     1
stat_Str          1
stat_Int          1
stat_Dex          1
stats_num         1
dtype: int64

### Commentary

이 Flic이라는 아이디를 쓰는 유저는 한 사람이다. twitch 아이디는 적혀있지 않지만 account가 한개이다. 

우연으로 겹친 아이디도 분명 있을 수도 있다. 

그런 유저들이 있는지, 같은 아이디이지만 다른 트위치 아이디를 가진 유저들을 찾아보자.

In [43]:
df['name'].value_counts().sort_values(ascending=False)

_Flic_            14
TundraDrum        11
LordTemplar       10
SirLalli           9
kotlecio           9
                  ..
ОгоньОчищения      1
AzertyTheWitch     1
pfooey             1
Politoese          1
Reselin            1
Name: name, Length: 58286, dtype: int64

In [44]:
df['twitch'].value_counts().sort_values(ascending=False)

thespunkylobster    18
iispasov            16
seyser_koze         16
yoshixt             15
egheo17             15
                    ..
jigalopoe            1
taggarn              1
ragnion              1
kimitsugo            1
lecarn               1
Name: twitch, Length: 9613, dtype: int64

한 twitch 아이디 안에 있는 여러 이름과 아이디들을 보자.

가장 많은 수를 기록했던 thespunkylobster 를 보도록 해보자.

In [37]:
# twitch = thespunkylobster

df[df['twitch'] == 'thespunkylobster']

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
487,122,True,False,ThroughThePrismOfSkooma,94,Raider,0be0d070caf5d4254fb2bd731c2c2b5fead2e1db19cbcf...,2806353782,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
495,124,True,False,Khajiit_Is_Innocent,94,Gladiator,07ba54024352737d8693d61794a1e21f7ff60d4a7c556f...,2798624220,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2
1026,257,True,False,LobstersOnSkooma,93,Slayer,05c0abe82318cd4fdc20142d73b4ab4af36f24b32c94bd...,2465132591,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2
2887,722,True,False,WindsOfSkooma,90,Raider,ce0931dea5c235a231be0487cd51aed0f82bf8737b1ce2...,1955389953,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
4965,1243,False,False,ForTheLoveOfSkomma,88,Slayer,263ea5b2089de5383dd9e0eff5d308746bc1a6c362044e...,1689357993,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2
5612,1405,True,False,SpacedOutOnSkomma,87,Pathfinder,8cb6391296e8b7916cc1174195bf261d9ac4103f7abab1...,1619835792,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
8675,2173,False,False,SkoomaRage,85,Necromancer,ddf286fe358060d8dc3eac59e61541051b774fee6ff127...,1335048935,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,1,0,1
8744,2190,True,False,SkoomaCausesBadDecision,85,Deadeye,dfa99a355be72cc79fe54b0312b623df367ed4a05f2cd5...,1328292006,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
9659,2419,False,False,DarthSkooma,84,Berserker,db7f3565e482bdd84e0589f7c6685ad7e78273f234a9d3...,1260634487,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,0,1
10299,2579,True,False,SkoomaMeatballs,84,Slayer,d9fdf5925af585ef933e992b17a31585ca483bbc3b5d83...,1213016381,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2


In [39]:
df[df['twitch'] == 'thespunkylobster'].nunique()

rank             18
dead              2
online            1
name             16
level            13
class             9
id               18
experience       18
account           1
challenges        1
twitch            1
ladder            1
special_class     1
stat_Str          2
stat_Int          2
stat_Dex          2
stats_num         2
dtype: int64

In [ ]:
df[df['twitch'] == 'thespunkylobster'].nunique()

### Commentary

name 을 보면 약간의 규칙이 있다. Skooma, Skomma, Lobster(s), Khajiit 등의 표현을 넣고 있으며, 그런 표현은 한 번만 등장하지 않는다. 

이를 보아서는 한 고수 유저가 최대 18개의 아이디를 관리하며 랭킹에 이름을 올렸을 가능성이 있다. 신기한 것은 이 유저는 SSF Harbinger HC 즉 혼자 하는 Hardcore 모드에만 등장한다는 점이다.